In [33]:
import re
import polars as pl
import altair as alt

In [8]:
pl.Config(tbl_rows=100)

In [5]:
df = pl.scan_parquet(
    '../zpravodajske-weby/data/dny/*.parquet'
).collect(
).sort(
    by='cas'
).with_columns(
    pl.col("titulek").str.replace(" nad ","_nad_").str.replace("České ","České_").str.replace("Staré ","Staré_").str.replace("Starého ","Starého_").str.replace("Nové ","Nové_").str.replace("Nového ","Nového_").str.replace("Moravské ","Moravské_").str.replace("Českých ","Českých_").str.replace("Moravských ","Moravských_").str.replace("Nebezpečná"," ").str.replace("KrimiMartin"," ")
).with_columns(
    pl.col('cas').dt.ordinal_day().alias('den')
).filter(
    pl.col("cas").dt.year() == 2025
)

In [49]:
", ".join(df.group_by("web").len().sort(by="len",descending=True).select(pl.col("web")).to_series().to_list())

'seznamzpravy.cz, blesk.cz, irozhlas.cz, denikn.cz, novinky.cz, denik.cz, idnes.cz, parlamentnilisty.cz, tn.nova.cz, aktualne.cz, lidovky.cz, echo24.cz, ceskenoviny.cz, extra.cz, forum24.cz, drbna.cz, zpravy.idnes.cz, cnn.iprima.cz'

In [7]:
def mesto(titulek, spojky=['z','ze']):
    titulek = re.sub(r'[^\w\s]', '', titulek)
    for s in spojky:
        if f" {s} " in titulek:
            fraze = f"{s} " + titulek.split(f' {s} ')[1].split(' ')[0]
            if fraze != fraze.lower():
                return fraze.replace("_"," ")

In [13]:
def najdi_vyskyty(popisek, retezec):
    return df.filter(
        pl.col("cas").dt.year() == 2025
    ).filter(
        pl.col("titulek").str.contains(retezec)
    ).unique(
        subset="titulek",keep='first'
    ).with_columns(
        pl.col('titulek').map_elements(mesto,return_dtype=pl.String).alias('mesto')
    ).group_by(
        'mesto'
    ).agg(
        pl.col('den').unique().len()
    ).sort(
        by='den',descending=True
    ).drop_nulls(
    ).with_columns(
        pl.lit(popisek).alias("co")
    )

In [81]:
do_grafu_df = pl.concat(
    [
        najdi_vyskyty("rekordman",r"(?i)rekordman(a|em|ovi|ům|y|i|ech|ům){0,1} ze{0,1} "),
        najdi_vyskyty("muž",r"(?i)muž(e|i|em|ích|ům|ovi|ů){0,1} ze{0,1} "),
        najdi_vyskyty("mladík",r"(?i)mladí(k|ka|kovi|ku|kem|ci|ky|ků|kům|kách|cích){0,1} ze{0,1} "),
        najdi_vyskyty("lidé",r"(?i)lid(é|í|mi|em|ech) ze{0,1} "),
        najdi_vyskyty("děti",r"(?i)dět(i|í|em|ech) ze{0,1} "),
        najdi_vyskyty("žena",r"(?i)žen(a|y|u|ě|ou|ami|ách|ám){0,1} ze{0,1} "),
        najdi_vyskyty("dívka",r"(?i)dív(ka|kou|ce|ek|ky|kám|kách|kami) ze{0,1} "),
        najdi_vyskyty("stařík","(?i)staří(k|ka|kovi|ku|kem|ci|ky|kům|kách|cích){0,1} ze{0,1} "),
        najdi_vyskyty("rodačka",r"(?i)rodač(ka|ky|kou|kách|ce|kami){0,1} ze{0,1} "),
        najdi_vyskyty("důchodce",r"(?i)důchodc(e|i|em|ovi|ům|ích){0,1} ze{0,1} "),
        najdi_vyskyty("senior",r"(?i)senio[rř](u|ovi|em|a|i|ům|ech|y){0,1} ze{0,1} "),
        najdi_vyskyty("studentka",r"(?i)student[kc](a|ou|e|ek|y|ám|ami|ách){0,1} ze{0,1} "),
        najdi_vyskyty("manželé",r"(?i)manžel(é|ům|y|ech|ové) ze{0,1} "),
        najdi_vyskyty("kamarádi",r"(?i)kamarád(i|ům|ech|ů) ze{0,1} "),
        najdi_vyskyty("student",r"(?i)student(ovi|a|i|ům|y|ech|em){0,1} ze{0,1} "),
        najdi_vyskyty("násilník",r"(?i)násilní[kc](a|em|u|ovi|i|y|ách|ích){0,1} ze{0,1} "),
        najdi_vyskyty("vědkyně",r"(?i)vědky[ňn](i|e|ě|ím|í|ěmi|ích){0,1} ze{0,1} "),
        najdi_vyskyty("vědec",r"(?i)věd(ec|ci|cům|cem|ců|ce|cích){1} ze{0,1} "),
        najdi_vyskyty("zahrádkář",r"(?i)zahrádkář(ovi|e|i|e|em|ům|ích){0,1} ze{0,1} "),
        najdi_vyskyty("houbař",r"(?i)houbař(i|e|em|ovi|ům|ích){0,1} ze{0,1} "),
        najdi_vyskyty("podivín",r"(?i)podivín(a|em|y|ech|i|ům|ovi){0,1} ze{0,1} "),
        najdi_vyskyty("bezdomovec",r"(?i)bezdomov(ec|ci|covi|ce|cem|ci|cům|cích){0,1} ze{0,1} "),
        najdi_vyskyty("seniorka",r"(?i)senior[kc](a|e|y|ou|ám|ách|ami){0,1} ze{0,1} "),
        najdi_vyskyty("důchodkyně",r"(?i)důchodk(yně|yni|yní|yněmi|yních){0,1} ze{0,1} "),
        najdi_vyskyty("senior",r"(?i)senio[rř](a|em|i|u|ovi|r|y|ům|ech){0,1} ze{0,1} "),
        najdi_vyskyty("školák",r"(?i)školá[kc](a|em|u|ovi|r|y|ům|ech|ích){0,1} ze{0,1} "),
        najdi_vyskyty("školačka",r"(?i)školač[kc](a|ou|y|ami|ách|ek){0,1} ze{0,1} "),
        najdi_vyskyty("přátelé",r"(?i)přátel(e|é|i|ům|ích|ech|y){0,1} ze{0,1} "),
        najdi_vyskyty("pár",r"(?i)[^\w]pár(em|y|u|ům|ech){0,1} ze{0,1} "),
        najdi_vyskyty("chlapec",r"(?i)chlap(ec|ci|cem|covi|cích|cům|ců) ze{0,1} "),
        najdi_vyskyty("výtržník",r"(?i)výtržník(a|c|y|en|ou|e|em|ovi|i|a|ou|y|ovi|ům|ech|y|e|i|í|ích){0,1} ze{0,1} "),
        najdi_vyskyty("podvodník",r"(?i)podvodník(a|em|ou|u|ovi|i|ům|ech|y|ích){0,1} ze{0,1} "),
        najdi_vyskyty("cyklista",r"(?i)cyklist(a|u|ou|y|i|ům|ech){0,1} ze{0,1} "),
        najdi_vyskyty("řidička",r"(?i)řidič[kc](e|a|ou|y|ám|ami|ách){0,1} ze{0,1} "),
        najdi_vyskyty("řidič",r"(?i)řidič(e|i|em|ovi|ům|ích|){0,1} ze{0,1} "),
        najdi_vyskyty("dealer",r"(?i)deale[rř](a|em|u|ovi|i|ům|ech|y|ích){0,1} ze{0,1} "),
        najdi_vyskyty("rodák",r"(?i)rodá[kc](a|em|u|ovi|ů|i|ům|y|ích){0,1} ze{0,1} "),
        najdi_vyskyty("lékař",r"(?i)lékař(e|i|em|u|ovi|ům|y|ů|ích){0,1} ze{0,1} "),
        najdi_vyskyty("doktor",r"(?i)dokto[rř](a|em|u|ovi|i|ech|ům|y|ích){0,1} ze{0,1} "),
        najdi_vyskyty("lékařka",r"(?i)lékař(ka|kou|ce|ky|kami|ek|kách){0,1} ze{0,1} "),
        najdi_vyskyty("doktorka",r"(?i)doktor(ka|kou|ce|ky|kami|ek|kách){0,1} ze{0,1} "),
    ]
)

In [82]:
nechceme = [
    'z ANO',
    'z Hoří',
    'z Hradu',
    'z Aljašky',
    'z Ukrajiny',
    'z Jemenu',
    'z ODS',
    'ze Světových',
    'z ČR',
    'ze Slovenska',
    'z Česka',
    'z Kyjeva',
    'z USA',
    'z US',
    'z Německa',
    'z Gazy'
]

do_grafu = do_grafu_df.filter(
    ~pl.col("mesto").is_in(nechceme)
).sort(by="den",descending=True)

vyber = do_grafu.group_by("mesto").agg(pl.col("den").sum()).sort(by="den",descending=True).head(25).select(pl.col("mesto")).to_series().to_list()

do_grafu = do_grafu.filter(pl.col("mesto").is_in(vyber))

filtr_cetnosti = (
    do_grafu
    .group_by("co")
    .agg(
        # We explicitly name this "pocet_mest"
        pl.col("mesto").unique().len().alias("pocet_mest") 
    )
    .filter(pl.col("pocet_mest") >= 3)
    .select(pl.col("co"))
    .to_series()
    .to_list()
)

do_grafu = do_grafu.filter(pl.col("co").is_in(filtr_cetnosti))

poradi_kategorii = do_grafu.group_by("co").agg(pl.col("den").sum()).sort(by="den",descending=True).select(pl.col("co")).to_series().to_list()

do_grafu

mesto,den,co
str,u32,str
"""z Brna""",26,"""muž"""
"""z Vysočiny""",19,"""rodák"""
"""z Brna""",10,"""vědec"""
"""z Prahy""",9,"""muž"""
"""ze Šumperska""",9,"""žena"""
"""z Karlovarska""",8,"""muž"""
"""z Plzeňska""",7,"""muž"""
"""ze Sokolovska""",6,"""muž"""
"""z Plzeňska""",6,"""žena"""


In [83]:
mrizka='lightgray'

base = alt.Chart(
    do_grafu,
    title=alt.Title(
        "V kolika různých dnech se loni objevili v titulcích na zpravodajských webech…",
        subtitle=[
            "Sledované stránky: seznamzpravy.cz, blesk.cz, irozhlas.cz, denikn.cz, novinky.cz, denik.cz,",
            "idnes.cz, parlamentnilisty.cz, tn.nova.cz, aktualne.cz, lidovky.cz, echo24.cz, ceskenoviny.cz,",
            "extra.cz, forum24.cz, drbna.cz, zpravy.idnes.cz (pouze od 1. 7.), cnn.iprima.cz"
        ]
    )
).encode(
    alt.Y("co:N", sort=poradi_kategorii, title=None, axis=alt.Axis(grid=True, gridColor=mrizka, tickColor=mrizka, tickOffset=0, tickWidth=0.5, gridWidth=0.5, zindex=0, domainColor="white")),
    alt.X("mesto:N", sort=vyber, title=None, axis=alt.Axis(orient='top', gridColor=mrizka, tickColor=mrizka, tickOffset=0, tickWidth=0.5, gridWidth=0.5,labelAngle=305, domainColor="white",grid=True, zindex=0)),
)

# 2. Create the rectangle layer
rects = base.mark_rect().encode(
    alt.Color("den:Q", scale=alt.Scale(range=['#FFFFE4',"#D6534B"], type="linear"), legend=None, title=None)
)

# 3. Create the text layer
# We map the "text" channel to the "len" column
text = base.mark_text(baseline='middle').encode(
    text='den:Q',
    # You can conditionally change color if the background is too dark
    color=alt.value('black') 
)

# 4. Combine them
chart = rects + text

chart.configure_view(stroke='transparent')

alt.LayerChart(...)